# For L7 epoch

In [1]:
import requests
import json
import numpy as np

# ['5','6','7','8','9'] = ["JUPITER BARYCENTER", "SATURN BARYCENTER", "URANUS BARYCENTER", "NEPTUNE BARYCENTER", "PLUTO BARYCENTER"]
planets = ['5','6','7','8','9']
orbital_elements = []
for planet in planets:
    # Construct the API URL
    # Date is 2004-06-01
    date  = '2004-06-01'
    edate = '2004-06-02'
    units = 'AU-D'
    
    url = f"https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='{planet}'&OBJ_DATA='YES'&MAKE_EPHEM='YES'&EPHEM_TYPE='ELEMENT'&CENTER='500@0'&START_TIME='{date}'&STOP_TIME='{edate}'&STEP_SIZE='1%20d'&CSV_FORMAT='YES&OUT_UNITS='{units}'"
    # Send an HTTP GET request to the API
    response = requests.get(url)
    
    # List: [EC, QR, IN, OM, W , Tp ,N , MA, TA ,A , AD, PR]
    #       [0   1   2   3   4   5   6   7   8   9   10  11]
    str = response.text

    #Get just the data from string ####consider using json instead####
    data = (((str.split('$$SOE\n')[1]).split('\n$$EOE')[0]).split('\n')[0][52:]).split(',')

    # Extract the orbital elements from the response
    a = eval(data[9]) # Semi-major axis (in AU)
    e = eval(data[0]) # Eccentricity
    inc = eval(data[2]) # Inclination (in degrees)
    Omega = eval(data[3]) # Longitude of the ascending node (in degrees)
    omega = eval(data[4]) # Argument of perihelion (in degrees)
    f = eval(data[8]) # True anomaly (in degrees)

    # Append the orbital elements to empty list
    orbital_elements.append({"a": a, "e": e, "inc": inc, "Omega": Omega, "omega": omega, "f": f})

# Convert degrees to radians
for elem in orbital_elements:
    elem["inc"] = np.deg2rad(elem["inc"])
    elem["omega"] = np.deg2rad(elem["omega"])
    elem["Omega"] = np.deg2rad(elem["Omega"])
    elem["f"] = np.deg2rad(elem["f"])

# For MPC

In [1]:
import requests
import json
import numpy as np

headers = ['JDTDB', 'Calendar Date (TDB)', 'EC', 'QR', 'IN', 'OM', 'W', 'Tp', 'N', 'MA', 'TA', 'A', 'AD', 'PR']
# ['5','6','7','8','9'] = ["JUPITER BARYCENTER", "SATURN BARYCENTER", "URANUS BARYCENTER", "NEPTUNE BARYCENTER", "PLUTO BARYCENTER"]
planets = ['5','6','7','8','9']
epochs = []
for planet in planets:
    orbital_elements = []
    # Construct the API URL
    # Date is 2004-06-01
    date  = '1994-9-24'
    edate = '2023-2-25'
    units = 'AU-D'
    step_size_hr=12
    
    url = f"https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='{planet}'&OBJ_DATA='YES'&MAKE_EPHEM='YES'&EPHEM_TYPE='ELEMENT'&CENTER='500@0'&START_TIME='{date}'&STOP_TIME='{edate}'&STEP_SIZE='{step_size_hr}%20h'&CSV_FORMAT='YES&OUT_UNITS='{units}'"
    # Send an HTTP GET request to the API
    response = requests.get(url)
    values_list = [value.split(',')[:-1] for value in (response.text.split('\n')[37:-66])]
    
    
    # List: [EC, QR, IN, OM, W , Tp ,N , MA, TA ,A , AD, PR]
    #       [0   1   2   3   4   5   6   7   8   9   10  11]
    for values in values_list:
        params = dict(zip(headers, values))
        orbital_elements.append(params)
    epochs.append(orbital_elements)


## Convert degrees to radians
#for elem in orbital_elements:
#    elem["inc"] = np.deg2rad(elem["inc"])
#    elem["omega"] = np.deg2rad(elem["omega"])
#    elem["Omega"] = np.deg2rad(elem["Omega"])
#    elem["f"] = np.deg2rad(elem["f"])

In [18]:
epochs[3][0]

{'JDTDB': '2449620.000000000',
 'Calendar Date (TDB)': ' A.D. 1994-Sep-24 12:00:00.0000',
 'EC': '  8.644393445029328E-03',
 'QR': '  2.980990514101379E+01',
 'IN': '  1.770105813810094E+00',
 'OM': '  1.317838124196468E+02',
 'W': '  2.735291933437573E+02',
 'Tp': '  2.468336836645805E+06',
 'N': '  5.980879473811010E-03',
 'MA': '  2.480568558904322E+02',
 'TA': '  2.471417780080189E+02',
 'A': '  3.006984067463468E+01',
 'AD': '  3.032977620825557E+01',
 'PR': '  6.019181653406708E+04'}

In [19]:
import csv

to_csv = epochs[3]

keys = to_csv[3].keys()

with open('Neptune.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(to_csv)